# Chapter 3: Unsupervised Learning and Customer Segmentation

From: https://www.amazon.com/Data-Science-Marketing-Analytics-practical/dp/1800560478 

Key concepts: 
-   Entities (customers, offers, etc)
-   Properties (age, income, etc)

Overall process:
1.  Chossing the relevant attributes for the entities
2.  Clustering/goruping the entities
3.  Understanding and describing the segments

#### Exercise 3.01: Mall customer segmentation - understanding the data

Get data from: https://packt.link/Dgl1z